<a href="https://colab.research.google.com/github/Katrin-Leberfinger/Hybrid-gender-debiased-music-recommendation/blob/main/03_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import torch
import numpy as np
import torch.nn as nn
from torch.autograd import Variable
import pandas as pd
import torch
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import gc                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

# Read Data

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
%cd /content/drive/MyDrive/Master\ Thesis/data

/content/drive/MyDrive/Master Thesis/data


## Read Data: Lyrics 

In [55]:
dir = "preprocessed_data_binary/"

data_interaction = pd.read_csv(dir + "data_interaction.csv").drop(['Unnamed: 0'],axis=1)
data_tracks_tags_lyrics = pd.read_csv(dir + "data_tracks_tags_lyrics.csv").drop(['Unnamed: 0'],axis=1)
data_interaction_test = pd.read_csv(dir + "data_interaction_test.csv").drop(['Unnamed: 0'],axis=1)
data_interaction_train = pd.read_csv(dir + "data_interaction_train.csv").drop(['Unnamed: 0'],axis=1)
data_interaction_val = pd.read_csv(dir + "data_interaction_val.csv").drop(['Unnamed: 0'],axis=1)

In [56]:
data_interaction_train = data_interaction_train.drop_duplicates(subset = ["user_id", "track_id", "count"])
data_interaction_test = data_interaction_test.drop_duplicates(subset = ["user_id", "track_id", "count"])
data_interaction_val = data_interaction_val.drop_duplicates(subset = ["user_id", "track_id", "count"])


In [57]:
pos2item = {i:v for i, v in enumerate(data_tracks_tags_lyrics.track_id.sort_values().unique())}
item2pos = {v:i for i, v in enumerate(data_tracks_tags_lyrics.track_id.sort_values().unique())}

id2user = {i:v for i, v in enumerate(data_interaction.user_id.sort_values().unique())}
user2id = {v:i for i, v in enumerate(data_interaction.user_id.sort_values().unique())}

# Ask Me Anything Rating

Code source: https://github.com/nlp-deepcbrs/amar

In [58]:
def get_coverage(df_rec, data_tracks_tags_lyrics):
  return len(df_rec['track_id'].drop_duplicates()) / len(data_tracks_tags_lyrics['track_id'].drop_duplicates())

In [59]:
def get_ndcg(rel_true, k):

  rel_opt = np.zeros(len(rel_true))
  rel_opt[:k] = 1
  #rel_opt = rel_opt[:k]
  #rel_true = rel_true[:k]

  def _dcg(rel):
    i = np.arange(1, len(rel)+ 1)
    denom = np.log2(i + 1)
    dcg = np.sum(rel / denom)
    return dcg

  return _dcg(rel_true) / _dcg(rel_opt)



## **ItemKNN Model**

In [60]:
df_ratings_train = data_interaction_train[['user_id', 'track_id', 'count']]
df_ratings_test = data_interaction_test[['user_id', 'track_id', 'count']]

In [61]:
df_ratings_train[df_ratings_train.duplicated()]

,user_id,track_id,count


In [62]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [63]:
# pivot ratings into movie features
matrix_train = df_ratings_train.pivot(
    index='track_id',
    columns='user_id',
    values='count'
).fillna(0)

sim_train = csr_matrix(matrix_train.values)

In [64]:
df_ratings_train.pivot(
    index='track_id',
    columns='user_id',
    values='count'
)

user_id,9,17,44,55,77,80,82,92,98,117,...,117718,117888,118182,118288,118292,118479,118940,119516,119740,119741
track_id,,,,,,,,,,,,,,,,,,,,,
108734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161168,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41623005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41626691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41633274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
df_ratings_train.drop_duplicates().shape

(271465, 3)

In [66]:
df_ratings_train.shape

(271465, 3)

In [67]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(sim_train)
_, indices = model_knn.kneighbors(matrix_train.iloc[0,:].values.reshape(1,-1), n_neighbors = 100)

# **ItemKNN**: Get predictions

In [68]:
query_index = 0
predictions = {}

for user in df_ratings_test.user_id.unique():
  distances = []
  indices = []
  tid = df_ratings_train[(df_ratings_train.user_id == user) & (df_ratings_train['count'] == 1)].sort_values(by = 'count', ascending=False).reset_index()['track_id']
  for t in tid:
    d, i = model_knn.kneighbors(matrix_train[matrix_train.index == t].values.reshape(1,-1), n_neighbors = 100)
    i = np.array(matrix_train.index)[i]
    distances.extend(list(d.squeeze()))
    indices.extend(list(i.squeeze()))

  items_to_ignore = df_ratings_train[df_ratings_train['user_id'] == user]['track_id'].values
  bool_ignore = [x not in items_to_ignore for x in indices]
  indices = np.array(indices)[bool_ignore]
  distances = np.array(distances)[bool_ignore]

  indices = indices[np.array(distances).argsort()[:100].tolist()].tolist()

  predictions[user] = []

  predictions[user].append(indices)


In [69]:
results = []
results_df = pd.DataFrame(columns = ['user_id', 'track_id', 'count'])

for user in predictions:
    user_prediction = predictions[user][0]
    for item in user_prediction:
        rating = 1.
        results_df = pd.concat((results_df, pd.DataFrame(data={'user_id':[user],'track_id':[item],'count':[rating]})))
        results.append([user, item, rating])

In [70]:
model_dir = f"./results/itemKNN/"
import os
os.makedirs(model_dir, exist_ok=True)

In [71]:
results_df.to_csv(model_dir + 'predictions.csv', encoding = 'utf-8-sig')

# **Most popular**

In [75]:
topn = 100

item_popularity_df = data_interaction.groupby('track_id')['count'].sum().sort_values(ascending=False).reset_index()

predictions = {}

for real_user_id in data_interaction_test.user_id.unique():
    predictions[real_user_id] = []
    items_to_ignore = data_interaction_train[data_interaction_train['user_id'] == real_user_id]['track_id'].values
    item_popularity_df_user = item_popularity_df#[~item_popularity_df['track_id'].isin(items_to_ignore)]
    predictions[real_user_id].append(item_popularity_df_user.head(topn)['track_id'].values.tolist())

results = []
results_df = pd.DataFrame(columns = ['user_id', 'track_id', 'count'])

for user in predictions:
    user_prediction = predictions[user][0]
    for item in user_prediction:
        rating = 1.
        results_df = pd.concat((results_df, pd.DataFrame(data={'user_id':[user],'track_id':[item],'count':[rating]})))
        results.append([user, item, rating])

In [79]:
model_dir = f"./results/mostPop/"
import os
os.makedirs(model_dir, exist_ok=True)

In [80]:
results_df.to_csv(model_dir + 'predictions.csv', encoding = 'utf-8-sig')

# **Random recommendations**

In [83]:
topn = 100
import random

predictions = {}

for real_user_id in data_interaction_test.user_id.unique():
    predictions[real_user_id] = []
    items_to_ignore = data_interaction_train[data_interaction_train['user_id'] == real_user_id]['track_id'].values
    item_df_user = data_interaction
    predictions[real_user_id].append(random.sample(set(item_df_user['track_id'].values), topn))

results = []
results_df = pd.DataFrame(columns = ['user_id', 'track_id', 'count'])

for user in predictions:
    user_prediction = predictions[user][0]
    for item in user_prediction:
        rating = 1.
        results_df = pd.concat((results_df, pd.DataFrame(data={'user_id':[user],'track_id':[item],'count':[rating]})))
        results.append([user, item, rating])

In [84]:
model_dir = f"./results/random/"
import os
os.makedirs(model_dir, exist_ok=True)

In [85]:
results_df.to_csv(model_dir + 'predictions.csv', encoding = 'utf-8-sig')

# Evaluate predictions

Scores: F1@10, ndcg@k, recall@k

In [81]:
topn=100

In [86]:
f1_scores = []
recall_scores = []
ndcg_scores = []

for user, df in results_df.groupby('user_id'):
  df = df.drop_duplicates(subset='track_id').head(topn)

  y_true_sorted = data_interaction_test.loc[data_interaction_test['user_id'] == user].sort_values('count', ascending=False).drop_duplicates()
  y_ndcg = pd.merge(df[['track_id']], y_true_sorted[['track_id', 'count']].drop_duplicates(), 'left').fillna(0)['count'].values
  y_true_df = pd.merge(data_tracks_tags_lyrics[['track_id']].drop_duplicates(), y_true_sorted[['track_id', 'count']].drop_duplicates(), 'left').fillna(0)
  y_ndcg[y_ndcg > 0] = 1
  y_true_df.loc[y_true_df['count'] > 0, 'count'] = 1
  y_true = y_true_df['count'].values

  y_pred_df = pd.merge(data_tracks_tags_lyrics[['track_id']].drop_duplicates(), df[['track_id', 'count']], 'left').fillna(0)
  y_pred_df.loc[y_pred_df['count'] > 0, 'count'] = 1
  y_pred = y_pred_df['count'].values

  if y_true.sum() >= 1:
    f1_scores.append(f1_score(y_true, y_pred))
    recall_scores.append(recall_score(y_true, y_pred))
    ndcg_scores.append(get_ndcg(y_ndcg, topn))

coverage = get_coverage(results_df, data_tracks_tags_lyrics)

print("F1 Score: ", np.mean(f1_scores))
print("NDCG Score: ", np.mean(ndcg_scores))
print("Coverage: , ", coverage)

F1 Score:  0.005284605647979036
NDCG Score:  0.003163857458642479
Coverage: ,  1.0
